In [3]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from collections import *

In [4]:
device = torch.device('cuda')

In [5]:
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

In [6]:
train_dataset = torchvision.datasets.MNIST(root = 'mnist_data/', 
                                          train = True, 
                                          transform = transforms.ToTensor(), 
                                          download = False)
test_dataset = torchvision.datasets.MNIST(root = 'mnist_data/', 
                                        train = False, 
                                        transform = transforms.ToTensor())

In [7]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                               batch_size=batch_size, 
                                               shuffle=False)

In [10]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=num_classes):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(OrderedDict([('conv1', nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2)), 
                                                 ('norm1', nn.BatchNorm2d(16)), 
                                                 ('relu1', nn.ReLU()), 
                                                 ('pool1', nn.MaxPool2d(kernel_size=2, stride=2))]))
        self.layer2 = nn.Sequential(OrderedDict([('conv2', nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2)), 
                                                 ('norm2', nn.BatchNorm2d(32)), 
                                                 ('relu2', nn.ReLU()), 
                                                 ('pool2', nn.MaxPool2d(kernel_size=2, stride=2))]))
        self.layer3 = nn.Linear(7*7*32, num_classes)
        
    def forward(self, x):
        out = self.layer2(self.layer1(x))
        out = self.layer3(out.reshape(batch_size, -1))
        return out

In [11]:
model = ConvNet().to(device)

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [13]:
total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i + 1) % 100 == 0:
            print('Epoch:', epoch, '\tStep:', i, '\tLoss:', loss.item())

TypeError: conv2d(): argument 'padding' (position 5) must be tuple of ints, not str

In [83]:
with torch.no_grad():
    correct = 0
    total = len(test_loader) * batch_size
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        
        _, pred = torch.max(outputs.detach(), 1)
        
        correct += (pred == labels).sum().item()
        
    print((correct/total))

0.9894


In [78]:
model.eval()

ConvNet(
  (layer1): Sequential(
    (conv1): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (norm1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu1): ReLU()
    (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu2): ReLU()
    (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Linear(in_features=1568, out_features=10, bias=True)
)